# _Experimentation on 100k Tweets: January 19, 2020_

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#!pip install seaborn

In [3]:
# import libraries
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import random
import os

# Matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

## _Load in data_

In [4]:
df = pd.read_json("100k_text.json", orient="split")

In [5]:
df[:5]

,full_text,clean_tweet,label
0,"On October 16th, one supporter will get a chan...",october supporter will chance watch debate wit...,verified
1,OMG I’ve been dying to tell you about the new ...,been dying tell about cozy collection pajamas ...,verified
2,The #SchumerStandard for filling #SCOTUS vacan...,schumer standard filling scotus vacancies,verified
3,RT @alicia_keysbr: When someone says #TeamAlic...,when someone says team alicia strongest alread...,verified
4,"“Whether it is because of distance, health pro...",whether because distance health problems work ...,verified


In [6]:
#!pip install spacy
#!python -m spacy download en_core_web_sm
tweet = df[3:4]["full_text"].iloc[0]
print(tweet)

RT @alicia_keysbr: When someone says #TeamAlicia is the strongest but we already know that 💁🏽 https://t.co/7MDBKkfYJo


In [8]:
import spacy
# import string library function  
import string  
    
# Storing the sets of punctuation in variable result  
puncs = string.punctuation

#stopwords = spacy.lang.en.stop_words.STOP_WORDS

#nlp = spacy.load("en_core_web_sm")

In [9]:
df[:10]["full_text"].apply(lambda x: len(x.split()))

0    19
1    53
2     7
3    16
4    35
5    23
6    21
7    40
8    17
9    17
Name: full_text, dtype: int64

In [10]:
len(df[df["full_text"].str.startswith("RT")])

10135

In [11]:
# create new column indicating if its a retweet or not
df["retweet"] = df["full_text"].str.startswith("RT")

In [12]:
# find all user mentions
df["user_mentions"] = df["full_text"].str.findall("@[\w]*")

In [13]:
# how many mentions were there?
df["num_mentions"] = df["user_mentions"].apply(lambda x: len(x))

In [14]:
# create a column that outputs users mention as string with , dividing multiple mentions
df["mentions_usernames"] = df["user_mentions"].apply(lambda x: ",".join([mention for mention in x]))

In [15]:
# create column of how many links there are
df["num_links"] = df["full_text"].str.findall(r'https?://[^\s<>"]+|www\.[^\s<>"]+').apply(lambda x: len(x))

In [16]:
#!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiment(text, sent):
    """
    Gets VADER sentiment for tweet.
    """
    return SentimentIntensityAnalyzer().polarity_scores(text)[sent]

In [28]:
#!pip install --upgrade tqdm
#!pip install --upgrade spacy
#!python -m spacy download en_core_web_md

In [134]:
from tqdm.notebook import tqdm
tqdm.pandas()

# create column with VADER pos score
#df["vader_compound"] = df["full_text"].progress_apply(lambda x: sentiment(x, "compound"))

In [142]:
#df.to_json("100k_tweetsjan19.json", orient="split")

In [34]:
!pip install wordsegment
from wordsegment import load, segment
load()

    100% |████████████████████████████████| 4.8MB 8.4MB/s eta 0:00:01


In [38]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")

In [ ]:
flat_list = [item for sublist in l for item in sublist]

In [108]:
def spacy_analyze(text, nlp):
    """
    Spacy analysis of Tweet.
    """
    text = re.sub(r"RT|&amp|\d+th|@[\w]*|http\S+|\n\n|\n", "", text).strip()
    doc = nlp(text)
    wordlist = [token.text for token in doc if not token.is_punct]
    word_len = np.mean([len(token.text) for token in doc if not token.is_punct])
    return len(wordlist), word_len, wordlist

In [109]:
def split_hashtag(string):
    return re.sub(r"#(\w+)", segment(r"#(\w+)"), string)
    
    
"""    
    fi = fo.findall(string)
    result = ''
    for var in fi:
        result += var + ' '
    print(result)
"""

"    \n    fi = fo.findall(string)\n    result = ''\n    for var in fi:\n        result += var + ' '\n    print(result)\n"

In [110]:
test = df["full_text"][2:3].iloc[0]

In [111]:
#split_hashtag(test)

In [112]:
df["full_text"][:20].apply(lambda x: spacy_analyze(x, nlp))

0     (18, 4.333333333333333, [On, October, one, sup...
1     (50, 3.88, [OMG, I, ’ve, been, dying, to, tell...
2     (6, 7.166666666666667, [The, SchumerStandard, ...
3     (14, 4.357142857142857, [When, someone, says, ...
4     (37, 4.972972972972973, [Whether, it, is, beca...
5     (23, 4.0, [What, you, are, saying, is, I, was,...
6     (20, 3.65, [QB, ➡, ️, QB, respect, and,  , sho...
7     (43, 4.046511627906977, [Men, lie, Women, lie,...
8     (16, 6.4375, [Momentum, energy, transfer, Elas...
9     (15, 4.2, [Bernie, Sanders, Walks, out, to, Yo...
10    (13, 4.230769230769231, [Powerball, numbers, d...
11    (45, 4.288888888888889, [The, odds, of, dying,...
12    (26, 3.6538461538461537, [I, could, not, be, m...
13                      (5, 2.2, [Day, ☝, ️, of, 2019])
14                              (2, 4.5, [Patience, ⏱])
15    (26, 3.5384615384615383, [Ron, is, on, a, miss...
16    (45, 5.044444444444444, [Betsy, DeVos, 's, own...
17    (36, 5.0, [The, president, 's, personal, n

In [70]:
# find out how many users were mentioned
df["full_text"][:20].str.findall("@[\w]*")

0                                     []
1                               [@skims]
2                                     []
3                       [@alicia_keysbr]
4                        [@NYTParenting]
5                           [@AndyYork8]
6     [@DangeRussWilson, @deshaunwatson]
7                       [@itsgabrielleu]
8                                     []
9                           [@TheSource]
10                                    []
11                         [@GlobalFund]
12                    [@jamielynnspears]
13                                    []
14                                    []
15                      [@uninterrupted]
16                                    []
17                                    []
18                                    []
19                                    []
Name: full_text, dtype: object

In [54]:
import re

def tweet_clean(text):
    """
    Function that preprocesses tweet text.
    """
    text = re.sub(r"RT|&amp", "", text)
    return text.strip()

In [55]:
tweet_clean(tweet)

'@alicia_keysbr: When someone says #TeamAlicia is the strongest but we already know that 💁🏽 https://t.co/7MDBKkfYJo'

In [50]:
# trying to eliminate just punctuation
"".join([token for token in tweet if token not in puncs])

'RT aliciakeysbr When someone says TeamAlicia is the strongest but we already know that 💁🏽 httpstco7MDBKkfYJo'

In [51]:
# list comprehension of 
[word for word in tweet.split()]

['RT',
 '@alicia_keysbr:',
 'When',
 'someone',
 'says',
 '#TeamAlicia',
 'is',
 'the',
 'strongest',
 'but',
 'we',
 'already',
 'know',
 'that',
 '💁🏽',
 'https://t.co/7MDBKkfYJo']